In [10]:
import os
import numpy as np
import shutil
from shutil import ignore_patterns
import stat
import glob

This script merges simulations of AMSOS into a single simulation. This can be used when the following is true:
* You ran a simulation that finished. You wanted to continue the simulation, so you used the final config files as initial config files for another simulation. This gave you multiple folders (where the sim is basically broken up into pieces).

The goal of this script is to merge those pieces

In [11]:
# Specify the sim folders in order
sim_folders = [
    "/Users/saadjansari/Documents/Projects/AMSOS/resultsSummit/Tactoids/n400_f3600_short_poly_stability",
    "/Users/saadjansari/Documents/Projects/AMSOS/resultsSummit/Tactoids/n400_f3600_short_poly_stability_c",
]

This is a large copy and paste operation but with a supervisor that monitors and has the power to change the index of the files (i.e the data files are labelled like *6.dat, *91.dat). The index in all the individual pieces starts with a 0, but when merging them, the index of the latter pieces with be offset the last index of the former piece.

**Example**:
if the indices of the pieces are: $[[0,1,2,3,4], [0,1,2,3], [0,1,2,3,4,5]]$.

The final indices of the merge will be $[[0,1,2,3,4],[5,6,7,8],[9,10,11,12,13,14]]$. And then these can be merged into a single folder.

**Note: We will ignore the first frame of all the pieces except the first one. This first is just a copy and the action really starts at index 1.**

In [12]:
# Specify file patterns to ignore and those to ignore just for the latter copy operations
ext_ignore = ["*.log","*.err","*.csv","*.png","*.pdf","*.jobscript",".DS_Store","*.mp4","*pvtp.pvd", "*.out"]

In [13]:
# Define an updated copytree function
def copytree(src, dst, symlinks = False, ignore = None):
    if not os.path.exists(dst):
        os.makedirs(dst)
        shutil.copystat(src, dst)
    lst = os.listdir(src)
    if ignore:
        excl = ignore(src, lst)
        lst = [x for x in lst if x not in excl]
    for item in lst:
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if symlinks and os.path.islink(s):
            if os.path.lexists(d):
                os.remove(d)
            os.symlink(os.readlink(s), d)
            try:
                st = os.lstat(s)
                mode = stat.S_IMODE(st.st_mode)
                os.lchmod(d, mode)
            except:
                pass # lchmod not available
        elif os.path.isdir(s):
            copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

def file_key(f):
    k = int( f.split("_")[-1].split(".")[0])
    return k

def fold_key(f):
    k = int( f.split("-")[-1])
    return k

def replace_key(f,nk):
    fnew = "".join( idx_files[0].split("_")[:-1] ) + "_" + str(nk) + '.' + idx_files[0].split("_")[-1].split(".")[-1]
    return fnew

def check_key_bound_folder( k, fold):
    fk = fold_key( fold)
    if k > fk:
        return True
    else:
        return False
    
def get_new_bound_folder( old_fold):
    upper = fold_key( old_fold)
    lower = int(old_fold.split("/result")[-1].split("-")[0])
    newlower = upper+1
    newupper = upper+1+ (upper-lower)
    new_fold = os.path.join( "/".join( old_fold.split("/")[:-1]), "result") + str(newlower) + "-" + str(newupper)
    return new_fold

def get_dstfile_with_key_offset( srcfile, dstpath, offset):
    
    old_key = file_key( srcfile)
    new_key = old_key + offset
    tag = "_".join(srcfile.split("/")[-1].split("_")[:-1])
    ext = srcfile.split(".")[-1]
    dstfile = os.path.join( dstpath, tag + "_" + str(new_key) + "." + ext)
    return dstfile

In [14]:
# Get path of first folder. This is where the merge will be created.
head_tail = os.path.split(sim_folders[0])

# Begin by creating a new folder
merge_fold = os.path.join( head_tail[0], 'merge')
if os.path.exists(merge_fold):
    shutil.rmtree(merge_fold)
os.mkdir( merge_fold)
print('Created merge folder: {}'.format(merge_fold))

for idx,src in enumerate( sim_folders):
    
    # For the first folder, just copy almost every thing except those things to ignore
    if idx == 0:
        if not os.path.exists(merge_fold) or not os.listdir(merge_fold):
            copytree(sim_folders[0], merge_fold, ignore=ignore_patterns(*ext_ignore))
        else:
            raise Exception('There is already stuff inside merge folder')
    
    # For the latter folders
    else:
        
        # Find last index in results of merge
        # find result folders inside the main result folder
        res_folders = glob.glob( os.path.join(merge_fold, 'result/result*-*'))
        res_folders = sorted( res_folders, key=fold_key)
        # search inside last folder
        idx_files = glob.glob( os.path.join(res_folders[-1], 'SylinderAscii*.dat'))
        idx_files = sorted( idx_files, key=file_key)
        max_key = max([file_key(i) for i in idx_files])
        
        # Search in src folder
        # find result folders inside the main result folder
        src_files = glob.glob( os.path.join(src, 'result/result*-*/*'))
        src_files = sorted( src_files, key=file_key)
        
        resdest = res_folders[-1]
        for fil in src_files:
            
            # if key=0, skip it
            if file_key( fil) == 0:
                continue
                
            # check if key exceeds folder bound. If yes, switch to new folder
            if check_key_bound_folder( max_key+file_key(fil), resdest):
                resdest = get_new_bound_folder( resdest)
                print('Generated new destination folder: {}'.format(resdest))
                os.mkdir( resdest)

            dstfile = get_dstfile_with_key_offset( fil, resdest, max_key)
            # copy_file to dst folder with key offset
#            print(fil)
#            print(dstfile)
            # copy each file from src results to merge results after updating its index
            shutil.copyfile( fil, dstfile)
        

Created merge folder: /Users/saadjansari/Documents/Projects/AMSOS/resultsSummit/Tactoids/merge
